In [ ]:
import glob
import sys
import os
import fnmatch

DTI_DATA_FROM = '/local/PPMI_DTI/PPMI/'  # Contains data downloaded from loni (64 dcm files per DTI scan)
DTI_NIFTI_DATA_TO = '/local/PPMI/raw/' # Where to store the images once we have converted them to Nifti
todo_folders = []
print('Scanning DTI folder for images..')
for root, dir, files in os.walk(DTI_DATA_FROM):
    dcom_files_inside = fnmatch.filter(files, "*.dcm")
    if len(dcom_files_inside) > 0:
        todo_folders.append({
            'folder': root.split(DTI_DATA_FROM)[1],
            'image_id': int(dcom_files_inside[0].split('.')[-2].split('_I')[-1]),
        })
print('Found %s dcm images to process' % len(todo_folders))
todo_folders[0]

In [ ]:
for i, f in enumerate(todo_folders):
    folder_to = os.path.join(DTI_NIFTI_DATA_TO, f['folder'])
    image_base = os.path.join(folder_to, 'I%d' % f['image_id'])
    if os.path.exists('%s.nii.gz' % image_base):
        print('[%d/%d] Image %d: ALREADY DONE' % (i+1, len(todo_folders), f['image_id']))
        continue
    try:
        os.makedirs(folder_to)
    except OSError:
        pass  # Directory already exists
    cmd = 'mrconvert {dcom_images} {to_nifti_file} -export_grad_fsl {bvecs} {bvals} && gzip {to_nifti_file}'.format(
        dcom_images=os.path.join(DTI_DATA_FROM, f['folder']),
        to_nifti_file='%s.nii' % image_base,
        bvecs='%s_bvecs' % image_base,
        bvals='%s_bvals' % image_base,
    )
    os.system(cmd)
    print('[%d/%d] Image %d: DONE [%s]' % (i+1, len(todo_folders), f['image_id'], f['folder']))